<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/EDA_Verteilung_und_Ausrei%C3%9Feranalyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import io, os, sys, setuptools, tokenize
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from datetime import datetime
from datetime import timedelta
from math import sqrt

In [3]:
# Größe der Plots festlegen
matplotlib.rcParams['figure.figsize'] = (16, 9)

In [4]:
# Allgemeine Angaben
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

In [5]:
# Pfad zur CSV-Datei auf Google Drive
name_Messwerte = 'Messdaten_CAMS_GHI.csv'
name_Vorhersage = 'Vorhersagedaten_CAMS_VarIdx.csv'

folder_import = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
folder_export = '/content/drive/My Drive/Colab_Notebooks/plot_daily_UVI/'

In [6]:
df_Vorhersage = pd.read_csv(folder_import + name_Vorhersage)
df_Vorhersage.set_index('Datetime', inplace=True)
df_Vorhersage.index = pd.to_datetime(df_Vorhersage.index)

In [7]:
df_Messwerte = pd.read_csv(folder_import + name_Messwerte)
df_Messwerte.set_index('Datetime', inplace=True)
df_Messwerte.index = pd.to_datetime(df_Messwerte.index)

df_Messwerte['Observation_period'] = pd.to_datetime(df_Messwerte['Observation_period'])
df_Messwerte['date'] = df_Messwerte['Observation_period'].dt.date

In [8]:
def remove_sza_between_80_and_90(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    result = pd.DataFrame()

    for datum, df_tag in df.groupby('date'):
        # Zeilen entfernen, in denen SZA zwischen 80° und 90° liegt (exklusiv)
        df_filtered = df_tag[~df_tag['SZA'].between(70, 100)]
        result = pd.concat([result, df_filtered], ignore_index=True)

    return result

In [9]:
df_Messwerte = remove_sza_between_80_and_90(df_Messwerte)

In [10]:
hour_bins = [3, 6, 9, 12, 15, 18, 21]  # Beispiel für mögliche Bins
hour_labels = ['03:00-06:00', '06:00-09:00', '09:00-12:00', '12:00-15:00', '15:00-18:00', '18:00-21:00']
df_Messwerte['Time_bins'] = pd.cut(df_Messwerte['Hour'], bins=hour_bins, labels=hour_labels, right=False)